In [ ]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
%run "/Formula1/includes/configuration"

In [ ]:
schema = "race_id int, race_year int, round int, circuit_id int, name String, date String, time String"
race_df = spark.read.format('csv').schema(schema).option('header',True).load(raw_folder_path)

race_df.show(3)

+-------+---------+-----+----------+--------------------+----------+--------+
|race_id|race_year|round|circuit_id|                name|      date|    time|
+-------+---------+-----+----------+--------------------+----------+--------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29|06:00:00|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05|09:00:00|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19|07:00:00|
+-------+---------+-----+----------+--------------------+----------+--------+
only showing top 3 rows



Transformation of race data

In [ ]:
from pyspark.sql.functions import to_timestamp,concat, current_timestamp,concat_ws
race_df_transformed = race_df.select('race_id','race_year', 'round', 'circuit_id', 'name', to_timestamp(concat_ws(" ",race_df.date,race_df.time)).alias('race_timestamp'), current_timestamp().alias('ingestion_date') )
race_df_transformed.show(3)

+-------+---------+-----+----------+--------------------+-------------------+--------------------+
|race_id|race_year|round|circuit_id|                name|     race_timestamp|      ingestion_date|
+-------+---------+-----+----------+--------------------+-------------------+--------------------+
|      1|     2009|    1|         1|Australian Grand ...|2009-03-29 06:00:00|2023-06-14 10:57:...|
|      2|     2009|    2|         2|Malaysian Grand Prix|2009-04-05 09:00:00|2023-06-14 10:57:...|
|      3|     2009|    3|        17|  Chinese Grand Prix|2009-04-19 07:00:00|2023-06-14 10:57:...|
+-------+---------+-----+----------+--------------------+-------------------+--------------------+
only showing top 3 rows



ingest race data in process folder with partition on race_year and in parquet format

In [ ]:
display(dbutils.fs.ls('/mnt/formularacedata/'))

path,name,size,modificationTime
dbfs:/mnt/formularacedata/presentation/,presentation/,0,0
dbfs:/mnt/formularacedata/processed/,processed/,0,0
dbfs:/mnt/formularacedata/raw/,raw/,0,0


In [ ]:
destination_file_path = f"{processed_folder_path}/races"

In [ ]:
race_df_transformed.write.mode('overwrite')\
    .partitionBy('race_year') \
    .format('parquet')\
    .option('path', destination_file_path) \
    .save()


In [ ]:
dbutils.notebook.exit("success")